<a href="https://colab.research.google.com/github/harishkumar1218/neural_collaborative_filtering_with_tensorflow/blob/main/NCF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, Multiply,Concatenate
from tensorflow.keras.optimizers.legacy import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
import scipy.sparse as sp
import tensorflow as tf
import multiprocessing
from time import time
import numpy as np
import heapq
import math

In [2]:
#GMF
class GMF:
  def init_normal(shape, name=None):
      return tf.random_normal(shape, stddev=0.01)

  def get_model(num_users, num_items, latent_dim, regs=[0, 0]):
      # Input variables
      user_input = Input(shape=(1,), dtype=tf.int32, name='user_input')
      item_input = Input(shape=(1,), dtype=tf.int32, name='item_input')

      MF_Embedding_User = Embedding(input_dim=num_users, output_dim=latent_dim, name='user_embedding',
                                    embeddings_initializer=init_normal, embeddings_regularizer=l2(regs[0]), input_length=1)
      MF_Embedding_Item = Embedding(input_dim=num_items, output_dim=latent_dim, name='item_embedding',
                                    embeddings_initializer=init_normal, embeddings_regularizer=l2(regs[1]), input_length=1)

      # Crucial to flatten an embedding vector!
      user_latent = Flatten()(MF_Embedding_User(user_input))
      item_latent = Flatten()(MF_Embedding_Item(item_input))

      # Element-wise product of user and item embeddings
      predict_vector = Multiply()([user_latent, item_latent])

      # Final prediction layer
      prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name='prediction')(predict_vector)

      model = Model(inputs=[user_input, item_input], outputs=prediction)

      return model

In [3]:
#MLP
class MLP:
  def init_normal(shape):
      return tf.random_normal(shape, stddev=0.01)

  def get_model(num_users, num_items, layers=[64,32,16,8], reg_layers=[0,0,0,0]):
      assert len(layers) == len(reg_layers)
      num_layer = len(layers)
      user_input = Input(shape=(1,), dtype=tf.int32, name='user_input')
      item_input = Input(shape=(1,), dtype=tf.int32, name='item_input')

      MLP_Embedding_User = Embedding(input_dim=num_users, output_dim=layers[0]//2, name='user_embedding',
                                    embeddings_initializer=init_normal, embeddings_regularizer=l2(reg_layers[0]), input_length=1)
      MLP_Embedding_Item = Embedding(input_dim=num_items, output_dim=layers[0]//2, name='item_embedding',
                                    embeddings_initializer=init_normal, embeddings_regularizer=l2(reg_layers[0]), input_length=1)

      user_latent = Flatten()(MLP_Embedding_User(user_input))
      item_latent = Flatten()(MLP_Embedding_Item(item_input))

      vector = Concatenate()([user_latent, item_latent])

      for idx in range(1, num_layer):
          layer = Dense(layers[idx], kernel_regularizer=l2(reg_layers[idx]), activation='relu', name='layer%d' %idx)
          vector = layer(vector)

      prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name='prediction')(vector)

      model = Model(inputs=[user_input, item_input], outputs=prediction)

      return model


In [13]:
class Evaluation:
    def __init__(self, model, testRatings, testNegatives, K):
        self.model = model
        self.testRatings = testRatings
        self.testNegatives = testNegatives
        self.K = K

    def evaluate_model(self, num_thread):
        hits, ndcgs = [], []
        if num_thread > 1:  # Multi-thread
            pool = multiprocessing.Pool(processes=num_thread)
            res = pool.map(self.eval_one_rating, range(len(self.testRatings)))
            pool.close()
            pool.join()
            hits = [r[0] for r in res]
            ndcgs = [r[1] for r in res]
            return hits, ndcgs
        # Single thread
        for idx in range(len(self.testRatings)):
            (hr, ndcg) = self.eval_one_rating(idx)
            hits.append(hr)
            ndcgs.append(ndcg)
        return hits, ndcgs

    def eval_one_rating(self, idx):
        rating = self.testRatings[idx]
        items = self.testNegatives[idx]
        u = rating[0]
        gtItem = rating[1]
        items.append(gtItem)
        # Get prediction scores
        map_item_score = {}
        users = np.full(len(items), u, dtype='int32')
        predictions = self.model.predict([users, np.array(items)],
                                          batch_size=100, verbose=0)
        for i in range(len(items)):
            item = items[i]
            map_item_score[item] = predictions[i]
        items.pop()

        # Evaluate top rank list
        ranklist = heapq.nlargest(self.K, map_item_score, key=map_item_score.get)
        hr = self.getHitRatio(ranklist, gtItem)
        ndcg = self.getNDCG(ranklist, gtItem)
        return hr, ndcg

    @staticmethod
    def getHitRatio(ranklist, gtItem):
        for item in ranklist:
            if item == gtItem:
                return 1
        return 0

    @staticmethod
    def getNDCG(ranklist, gtItem):
        for i in range(len(ranklist)):
            item = ranklist[i]
            if item == gtItem:
                return math.log(2) / math.log(i + 2)
        return 0



In [5]:
class Dataset(object):
    def __init__(self, path):
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        self.num_users, self.num_items = self.trainMatrix.shape

    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList

    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1:]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList

    def load_rating_file_as_matrix(self, filename):
        '''
        Read .rating file and Return dok matrix.
        The first line of .rating file is: num_users\t num_items
        '''
        # Get number of users and items
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()

        # Construct matrix
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()
        return mat




In [6]:
#NeuMF
def get_model(num_users, num_items, mf_dim=10, layers=[10], reg_layers=[0], reg_mf=0):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP

    # Input variables
    user_input = Input(shape=(1,), dtype=tf.int32, name='user_input')
    item_input = Input(shape=(1,), dtype=tf.int32, name='item_input')

    # Embedding layer
    MF_Embedding_User = Embedding(input_dim=num_users, output_dim=mf_dim, name='mf_embedding_user',
                                  embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01),
                                  embeddings_regularizer=l2(reg_mf), input_length=1)
    MF_Embedding_Item = Embedding(input_dim=num_items, output_dim=mf_dim, name='mf_embedding_item',
                                  embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01),
                                  embeddings_regularizer=l2(reg_mf), input_length=1)

    MLP_Embedding_User = Embedding(input_dim=num_users, output_dim=layers[0]//2, name='mlp_embedding_user',
                                   embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01),
                                   embeddings_regularizer=l2(reg_layers[0]), input_length=1)
    MLP_Embedding_Item = Embedding(input_dim=num_items, output_dim=layers[0]//2, name='mlp_embedding_item',
                                   embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01),
                                   embeddings_regularizer=l2(reg_layers[0]), input_length=1)

    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = Multiply()([mf_user_latent, mf_item_latent]) # element-wise multiply

    # MLP part
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = Concatenate()([mlp_user_latent, mlp_item_latent])
    for idx in range(1, num_layer):
        layer = Dense(layers[idx], kernel_regularizer=l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate()([mf_vector, mlp_vector])

    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name="prediction")(predict_vector)

    model = Model(inputs=[user_input, item_input], outputs=prediction)

    return model

def load_pretrain_model(model, gmf_model, mlp_model, num_layers):
    # MF embeddings
    gmf_user_embeddings = gmf_model.get_layer('user_embedding').get_weights()
    gmf_item_embeddings = gmf_model.get_layer('item_embedding').get_weights()
    model.get_layer('mf_embedding_user').set_weights(gmf_user_embeddings)
    model.get_layer('mf_embedding_item').set_weights(gmf_item_embeddings)

    # MLP embeddings
    mlp_user_embeddings = mlp_model.get_layer('user_embedding').get_weights()
    mlp_item_embeddings = mlp_model.get_layer('item_embedding').get_weights()
    model.get_layer('mlp_embedding_user').set_weights(mlp_user_embeddings)
    model.get_layer('mlp_embedding_item').set_weights(mlp_item_embeddings)

    # MLP layers
    for i in range(1, num_layers):
        mlp_layer_weights = mlp_model.get_layer('layer%d' %i).get_weights()
        model.get_layer('layer%d' %i).set_weights(mlp_layer_weights)

    # Prediction weights
    gmf_prediction = gmf_model.get_layer('prediction').get_weights()
    mlp_prediction = mlp_model.get_layer('prediction').get_weights()
    new_weights = np.concatenate((gmf_prediction[0], mlp_prediction[0]), axis=0)
    new_b = gmf_prediction[1] + mlp_prediction[1]
    model.get_layer('prediction').set_weights([0.5*new_weights, 0.5*new_b])
    return model

def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j)in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels


In [7]:
#python NeuMF.py --dataset ml-1m --epochs 20 --batch_size 256 --num_factors 8 --layers [64,32,16,8] --reg_mf 0 --reg_layers [0,0,0,0] --num_neg 4 --lr 0.001 --learner adam --verbose 1 --out 1
num_epochs = 5
batch_size = 256
mf_dim = 8
layers = [64,32,16,8]
reg_mf = 0
reg_layers = [0,0,0,0]
num_negatives = 4
learning_rate = 0.001
learner = "adam"
verbose = 1
mf_pretrain = ''
mlp_pretrain = ''
save_model=1
topK = 10
evaluation_threads = 3 #mp.cpu_count()
model_out_file = f'/content/sample_data/model{time()}.h5'


In [8]:
# Loading data
t1 = time()
dataset = Dataset("/content/drive/MyDrive/neural_collaborative_filtering-master/Data/ml-1m")
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d"
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

# Build model
model = get_model(num_users, num_items, mf_dim, layers, reg_layers, reg_mf)
if learner.lower() == "adagrad":
    model.compile(optimizer=Adagrad(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "rmsprop":
    model.compile(optimizer=RMSprop(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "adam":
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy')

Load data done [16.9 s]. #user=6040, #item=3706, #train=994169, #test=6040


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
# Load pretrain model
if mf_pretrain != '' and mlp_pretrain != '':
    gmf_model = GMF.get_model(num_users,num_items,mf_dim)
    gmf_model.load_weights(mf_pretrain)
    mlp_model = MLP.get_model(num_users,num_items, layers, reg_layers)
    mlp_model.load_weights(mlp_pretrain)
    model = load_pretrain_model(model, gmf_model, mlp_model, len(layers))
    print(f'Load pretrained GMF {mf_pretrain} and MLP {mlp_pretrain} models done.' )

In [ ]:
# Init performance
(hits, ndcgs) = Evaluation(model, testRatings, testNegatives, topK).evaluate_model(evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f' % (hr, ndcg))
best_hr, best_ndcg, best_iter = hr, ndcg, -1
if save_model > 0:
    model.save_weights(model_out_file, overwrite=True)

In [ ]:
# Training model
for epoch in range(num_epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)

    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                      np.array(labels), # labels
                      batch_size=batch_size, verbose=0, shuffle=True,epochs=1)
    t2 = time()

    # Evaluation
    if epoch %verbose == 0:
        (hits, ndcgs) =Evaluation(model, testRatings, testNegatives, topK).evaluate_model(evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]'
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            if save_model > 0:
                model.save_weights(model_out_file, overwrite=True)

print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
if save_model > 0:
    print("The best NeuMF model is saved to %s" %(model_out_file))

Iteration 0 [163.6 s]: HR = 0.5975, NDCG = 0.3409, loss = 0.3165 [534.9 s]
Iteration 1 [116.2 s]: HR = 0.6354, NDCG = 0.3707, loss = 0.2718 [557.1 s]
Iteration 2 [164.6 s]: HR = 0.6490, NDCG = 0.3811, loss = 0.2625 [565.1 s]
Iteration 3 [162.7 s]: HR = 0.6603, NDCG = 0.3877, loss = 0.2577 [576.6 s]
Iteration 4 [117.5 s]: HR = 0.6639, NDCG = 0.3941, loss = 0.2542 [572.5 s]
End. Best Iteration 4:  HR = 0.6639, NDCG = 0.3941. 
The best NeuMF model is saved to /content/sample_data/model1711217973.1932163.h5


In [ ]:
import tensorflow as tf

# Load the model
model = get_model(num_users, num_items, mf_dim, layers, reg_layers, reg_mf)
model.load_weights("/content/sample_data/model17112179731932163.h5")


# Prepare input data for prediction (replace this with your actual input data)
# input_data = ""

# # Make predictions
predictions = model.predict([np.array([22]),np.array([934])])



print(predictions[0][0]*100)


1/1 [==============================] - 0s 124ms/step
20.198437571525574
